# Introduction to Semantic Kernel

Semantic Kernel is an AI framework similar to langchain & co. developed by Microsoft.

In this notebook we will look at the basic workings of the library.

In [ ]:
# Setup - call this once at the beginning of the notebook to add the parent directory to the path

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

## Setting up the Kernel

In Semantic Kernel all interactions with a LLM happens through a kernel.
Let's connect to it!